In [ ]:
!pip install -q streamlit

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
directory = "./gdrive/My Drive/style-transfer"


In [ ]:
%%writefile app.py

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import streamlit as st
from tensorflow.keras.applications import vgg19
import matplotlib.pyplot as plt
from PIL import Image as PILImage


def NST(content_img , style_img):

    directory = "./gdrive/My Drive/style-transfer"

    base_image_path = content_img
    style_reference_image_path = style_img
    content_image = keras.utils.load_img(base_image_path)
    style_image = keras.utils.load_img(style_reference_image_path)

    width, height = tf.keras.utils.load_img(base_image_path).size
    img_nrows = 400
    img_ncols = int(width * img_nrows / height)

    def preprocess_image(image_path):
        img = keras.utils.load_img(image_path, target_size=(img_nrows, img_ncols))
        img = keras.utils.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = vgg19.preprocess_input(img)
        return tf.convert_to_tensor(img)

    def deprocess_image(x):
        x = x.reshape((img_nrows, img_ncols, 3))
        x[:, :, 0] += 103.939
        x[:, :, 1] += 116.779
        x[:, :, 2] += 123.68
        x = x[:, :, ::-1]
        x = np.clip(x, 0, 255).astype("uint8")
        return x

    def gram_matrix(x):
        x = tf.transpose(x, (2, 0, 1))
        features = tf.reshape(x, (tf.shape(x)[0], -1))
        gram = tf.matmul(features, tf.transpose(features))
        return gram

    def style_loss(style, combination):
        S = gram_matrix(style)
        C = gram_matrix(combination)
        channels = 3
        size = img_nrows * img_ncols
        return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels**2) * (size**2))

    def content_loss(base, combination):
        channels = 3
        size = img_nrows * img_ncols
        return tf.reduce_sum(tf.square(combination - base)/ (4.0 * (channels) * (size)))

    def total_variation_loss(x):
        a = tf.square(x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :])
        b = tf.square(x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :])
        return tf.reduce_sum(tf.pow(a + b, 1.25))

    model = vgg19.VGG19(weights="imagenet", include_top=False)

    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

    for layer in model.layers:
        print (layer.name)


    feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

    style_layer_names = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
    content_layer_name = "block5_conv2"

    total_variation_weight = 1e-6
    style_weight = 1e-6
    content_weight = 2.5e-8

    def compute_loss(combination_image, base_image, style_reference_image):
        input_tensor = tf.concat([base_image, style_reference_image, combination_image], axis=0)
        features = feature_extractor(input_tensor)
        loss = tf.zeros(shape=())
        layer_features = features[content_layer_name]
        base_image_features = layer_features[0, :, :, :]
        combination_features = layer_features[2, :, :, :]
        loss = loss + content_weight * content_loss(base_image_features, combination_features)
        for layer_name in style_layer_names:
            layer_features = features[layer_name]
            style_reference_features = layer_features[1, :, :, :]
            combination_features = layer_features[2, :, :, :]
            sl = style_loss(style_reference_features, combination_features)
            loss += (style_weight / len(style_layer_names)) * sl
        loss += total_variation_weight * total_variation_loss(combination_image)
        return loss


    @tf.function
    def compute_loss_and_grads(combination_image, base_image, style_reference_image):
        with tf.GradientTape() as tape:
            loss = compute_loss(combination_image, base_image, style_reference_image)
        grads = tape.gradient(loss, combination_image)
        return loss, grads


    base_image = preprocess_image(base_image_path)
    style_reference_image = preprocess_image(style_reference_image_path)
    combination_image = tf.Variable(preprocess_image(base_image_path))


    optimizer = keras.optimizers.SGD(keras.optimizers.schedules.ExponentialDecay(
                       initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96))


    result_prefix = "stylised"
    iterations = 2500

    for i in range(1, iterations + 1):
        loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image )
        optimizer.apply_gradients([(grads, combination_image)])
        if i % 100 == 0:
            print("Iteration %d: loss=%.2f" % (i, loss))
            img = deprocess_image(combination_image.numpy())
            fname = "stylised_image.png"
            keras.utils.save_img(fname , img)

    stylised_image_path = os.path.join(directory, "stylised_image.png")
    PILImage.open("stylised_image.png").save(stylised_image_path)

st.title("Neural Style Transfer")
st.markdown("""
Welcome to the **Neural Style Transfer**! Blending Art and Technology for Creative Images.
Upload your **Content Image** and **Style Image** below, then click the 'Style' button to create your own masterpiece!
""")

def save_uploaded_file(uploaded_file, filename):
    try:
        with open(filename, "wb") as f:
            f.write(uploaded_file.getbuffer())
        return True
    except Exception as e:
        print(e)
        return False

content_img = st.file_uploader("Upload Content Image", type=["png", "jpg", "jpeg"])
if content_img is not None:
    if save_uploaded_file(content_img, "content_img.png"):
        st.image(content_img, caption="Content Image", use_column_width=True)

style_img = st.file_uploader("Upload Style Image", type=["png", "jpg", "jpeg"])
if style_img is not None:
    if save_uploaded_file(style_img, "style_img.png"):
        st.image(style_img, caption="Style Image", use_column_width=True)

if st.button("Style"):
    st.write("Styling in progress...wait for 10-15 minutes")
    NST('content_img.png','style_img.png')
    st.write("Here is your Masterpiece....")

    styled_image = keras.utils.load_img(os.path.join("./gdrive/My Drive/style-transfer", "stylised_image.png"))
    st.image(styled_image, caption="Styled Image", use_column_width=True)




In [ ]:
!npm install localtunnel

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501